In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import json
import random
import numpy as np
import pandas as pd
import io
from tqdm import tqdm
from icecream import ic
from import_casa import casa, cano
from casa.annot import AspectEnum

/Users/seantyh/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
TARGET_DIR = "20210412"
result_dir = casa.get_data_path() / f"annot_data/annotated_data_bkup/{TARGET_DIR}"
json_paths = [x for x in result_dir.iterdir() if x.suffix==".json"]
json_paths.sort()

In [4]:
with json_paths[1].open("r", encoding="UTF-8") as fin:
    annots = json.load(fin)

In [5]:
list(annots[0].keys())

['completions', 'data', 'id']

In [6]:
annot_x = annots[200]
aspects = cano.process_thread_annotations(annot_x)

In [7]:
aspects

[<AnnotAspect (std), 1-256, Thread 14887 :中華電信/[通訊]網速/完全正常/4>,
 <AnnotAspect (std), 1-256, Thread 14887 :中華電信/[其他]手機/完全正常/4>]

In [9]:
cano.make_proofread(aspects, annot_x["data"]["html"])

[{'annot_type': 'Entity',
  'norm_val': '中華電信',
  'spantext': '中華',
  'rawtext': '完全正常,手機跟固網都中華的',
  'batch_idx': 1,
  'thread_idx': 14887,
  'serial': 256,
  'cursor': 'ol[1]/li[1]/text()',
  'span_id': 'heHUbAdeiU'},
 {'annot_type': 'Attribute',
  'norm_val': '[通訊]網速',
  'spantext': '固網',
  'rawtext': '完全正常,手機跟固網都中華的',
  'batch_idx': 1,
  'thread_idx': 14887,
  'serial': 256,
  'cursor': 'ol[1]/li[1]/text()',
  'span_id': 'ZF9mVmL8mj'},
 {'annot_type': 'Attribute',
  'norm_val': '[其他]手機',
  'spantext': '手機',
  'rawtext': '完全正常,手機跟固網都中華的',
  'batch_idx': 1,
  'thread_idx': 14887,
  'serial': 256,
  'cursor': 'ol[1]/li[1]/text()',
  'span_id': 'OSnxZqPvUZ'}]

## Main conversion loop

In [10]:
rec_list = []
ex_log = io.StringIO()
for json_path in tqdm(json_paths):
    with json_path.open("r", encoding="UTF-8") as fin:
        annots = json.load(fin)
    for annot_i, annot_x in enumerate(annots): 
        try:
            aspects = cano.process_thread_annotations(annot_x)
            records = cano.make_proofread(aspects, annot_x["data"]["html"])
            rec_list.extend(records)
        except:
            pass

 20%|██        | 2/10 [00:00<00:03,  2.48it/s]

約翰是哪裡?
不確定是什麼
多tag


 30%|███       | 3/10 [00:01<00:02,  2.45it/s]

可以算涵蓋嗎...?
地區+時段
metaphorical?
不確定指哪裡
安平區
台北市區
499之亂後
NP
日本國際漫遊
成功嶺


 40%|████      | 4/10 [00:01<00:02,  2.63it/s]

海景
sarcasm


100%|██████████| 10/10 [00:02<00:00,  4.19it/s]

test
test


In [11]:
len(rec_list)

3175

In [13]:
proof_df = pd.DataFrame.from_dict(rec_list)

In [18]:
proof_df = proof_df.loc[proof_df.norm_val.str.len() > 0, :]
proof_df.reset_index(inplace=True)
out_path = result_dir / f"aspects_proof_read_{TARGET_DIR}.csv"
proof_df.to_csv(out_path)

In [19]:
out_path

PosixPath('/Users/seantyh/langon/casa/data/annot_data/annotated_data_bkup/20210412/aspects_proof_read_20210412.csv')